In [21]:
import tensorflow as tf
import pandas as pd
import numpy as np

import time

In [125]:
df = pd.read_csv("../data/heart.csv")

In [126]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
df['famhist']=lb.fit_transform(df['famhist'])

In [127]:
y = df['chd']

df = df.drop('chd', 1)

In [128]:
y = np.array([y, -(y-1)]).T

In [129]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df, y, test_size=0.31, random_state=42)

In [130]:
for i in [X_train, X_test, y_train, y_test]:
    print(i.shape)

(318, 9)
(144, 9)
(318, 2)
(144, 2)


In [131]:
X_train=X_train.as_matrix()
y_train=y_train.as_matrix()
X_test=X_test.as_matrix()
y_test=y_test.as_matrix()

AttributeError: 'numpy.ndarray' object has no attribute 'as_matrix'

In [147]:
# Define paramaters for the model
learning_rate = 0.0001
batch_size = 6
n_epochs = 20
num_c = 2
num_f = X_train.shape[1]

X = tf.placeholder(tf.float32, [batch_size, num_f], name = "features")

Y = tf.placeholder(tf.float32, [batch_size, num_c], name="label")


# Step 3: create weights and bias
# weights and biases are initialized to 0
# shape of w depends on the dimension of X and Y so that Y = X * w + b
# shape of b depends on Y

w = tf.Variable(tf.random_normal(shape = [num_f, num_c], stddev= 0.01), name='weights')
b = tf.Variable(tf.zeros(shape = [1, num_c]), name='bias')
# Step 4: build model
# the model that returns the logits.
# this logits will be later passed through softmax layer
# to get the probability distribution of possible label of the image
# DO NOT DO SOFTMAX HERE
logits = tf.matmul(X, w) + b

# Step 5: define loss function
# use cross entropy loss of the real labels with the softmax of logits
# use the method:
# tf.nn.softmax_cross_entropy_with_logits(logits, Y)
# then use tf.reduce_mean to get the mean loss of the batch


with tf.name_scope('loss'):
    entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name="loss")
    loss = tf.reduce_mean(entropy)
tf.summary.scalar('loss', loss)

# Merge all the summaries and write them out to /tmp/mnist_logs (by default)
merged = tf.summary.merge_all()

# Step 6: define training op
# using gradient descent to minimize loss
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [149]:
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('../my_graphs/heart', sess.graph)
    # n_batches = int(X_train.shape[0]/batch_size)
    for i in range(n_epochs): # train the model n_epochs times
        total_loss = 0
        n_batches = int(X_train.shape[0]/batch_size)
        X_batches = np.array_split(X_train, n_batches)
        Y_batches = np.array_split(y_train, n_batches)
        # Loop over all batches

        for i in range(n_batches):
            batch_x, batch_y = X_batches[i], Y_batches[i]
            print(np.dtype(batch_x[0]))
            _, loss_batch = sess.run([optimizer, loss], feed_dict={X: batch_x, Y: batch_y})
            # writer.add_summary(merged, _)
            total_loss += loss_batch
            print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

    print('Total time: {0} seconds'.format(time.time() - start_time))

    print('Optimization Finished!') # should be around 0.35 after 25 epochs

    # test the model
    preds = tf.nn.softmax(logits)
    correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # need numpy.count_nonzero(boolarr) :(
    
    # n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    total_batch = X_test.shape[0]/batch_size
    X_batches = np.array_split(X_test, total_batch)
    Y_batches = np.array_split(y_test, total_batch)
    
    for i in range(total_batch):
        # X_batch, Y_batch = mnist.test.next_batch(batch_size)
        batch_x, batch_y = X_batches[i], Y_batches[i]
        accuracy_batch = sess.run([accuracy], feed_dict={X: batch_x, Y:batch_y})
        total_correct_preds += accuracy_batch[0]

    print('Accuracy {0}'.format(total_correct_preds/X_test.shape[0]))
    writer.close()

TypeError: data type not understood

In [141]:
# Parameters
learning_rate = 0.001
training_epochs = 10
batch_size = 6
display_step = 5

# Network Parameters
n_hidden_1 = 10 # 1st layer number of features
n_hidden_2 = 10 # 2nd layer number of features
n_input = X_train.shape[1] # Number of feature
n_classes = 2 # Number of classes to predict

# tf Graph input
X = tf.placeholder(tf.float32, [batch_size, num_f], name = "features")

Y = tf.placeholder(tf.float32, [batch_size, num_c], name="label")

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [142]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# Initializing the variables
# init = tf.global_variables_initializer()

In [150]:
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('../my_graphs/heart', sess.graph)
    # n_batches = int(X_train.shape[0]/batch_size)
    for i in range(n_epochs): # train the model n_epochs times
        total_loss = 0
        n_batches = int(X_train.shape[0]/batch_size)
        X_batches = np.array_split(X_train, n_batches)
        Y_batches = np.array_split(y_train, n_batches)
        
        # X_batches = tf.constant(X_batches)
        # Loop over all batches
        for i in range(n_batches):
            batch_x, batch_y = X_batches[i], Y_batches[i]
            batch_x= tf.convert_to_tensor(batch_x, dtype=tf.float32)
            _, loss_batch = sess.run([optimizer, loss], feed_dict={X: batch_x, Y: batch_y})
            # writer.add_summary(merged, _)
            total_loss += loss_batch
            print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

    print('Total time: {0} seconds'.format(time.time() - start_time))

    print('Optimization Finished!') # should be around 0.35 after 25 epochs

    # test the model
    preds = tf.nn.softmax(logits)
    correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # need numpy.count_nonzero(boolarr) :(
    
    # n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    total_batch = X_test.shape[0]/batch_size
    X_batches = np.array_split(X_test, total_batch)
    Y_batches = np.array_split(y_test, total_batch)
    
    for i in range(total_batch):
        # X_batch, Y_batch = mnist.test.next_batch(batch_size)
        batch_x, batch_y = X_batches[i], Y_batches[i]
        accuracy_batch = sess.run([accuracy], feed_dict={X: batch_x, Y:batch_y})
        total_correct_preds += accuracy_batch[0]

    print('Accuracy {0}'.format(total_correct_preds/X_test.shape[0]))
    writer.close()

TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles.

In [136]:
y_test.shape

(144, 2)